In [ ]:
# define the model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, GlobalAveragePooling2D, Activation, Add, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical
import numpy as np

In [ ]:
img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

aesthetic_layer = Sequential([
    Dense(256, activation='relu', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1)
])

score_a = aesthetic_layer(img_feature_a)
score_b = aesthetic_layer(img_feature_b)

negated_score_b = Lambda(lambda x: -x, output_shape=(1,))(score_b)
diff = Add()([score_a, negated_score_b])

output = Activation("sigmoid")(diff)

ranknet = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

ranknet.load_weights("./data/ranknet/ranknet.h5")

##########################################################################################################################

base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x) # add a global spatial average pooling layer
x = aesthetic_layer(x)
aesthetic_model = Model(inputs=base_model.input, outputs=x)

aesthetic_model.summary()

In [ ]:
def img_preprocess(imgfile):
    try:
        img = image.load_img(imgfile, target_size=(299, 299))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        return x
    except:
        return None

def sigmoid(x):
    return 1.0/(1+np.exp(-x))

def get_aesthetic_score(imgfile):
    x = img_preprocess(imgfile)
    score = aesthetic_model.predict(np.array([x]))
    return sigmoid(score)

def get_aesthetic_score_on_batch(imgfiles):
    X = []
    valid_imgfiles = []
    for imgfile in imgfiles:
        x = img_preprocess(imgfile)
        if x is None:
            continue
        X.append(x)
        valid_imgfiles.append(imgfile)
    scores = aesthetic_model.predict(np.array(X))
    return valid_imgfiles, sigmoid(scores)

In [ ]:
# test1
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
from os.path import basename, splitext
import random

def Plot(imgfile, score):
    imgid = splitext(basename(imgfile))[0]
    
    plt.figure()
    
    plt.title("%s_%s"%(imgid, score))
    img = Image.open(imgfile)
    plt.imshow(img)
    plt.axis('off')
    
    plt.show()

imgids_by_date = {}
with open("../data/img_attr.csv") as fin:
    for line in fin:
        attrs = line.strip().split("\t")
        if len(attrs) != 7:
            continue
        imgid = attrs[0]
        date = attrs[5][:8]
        imgids_by_date.setdefault(date, [])
        imgids_by_date[date].append(imgid)

In [ ]:
curdate = "2017/04/"
imgfiles = map(lambda x: "../data/img/%s.jpg"%x, imgids_by_date[curdate])
imgfiles, imgscores = get_aesthetic_score_on_batch(imgfiles)
img2score = sorted(zip(imgfiles, imgscores), key=lambda x: x[1], reverse=True)

In [ ]:
for imgfile, score in img2score[:10]:
    Plot(imgfile, np.round(score*100))
    
for imgfile, score in img2score[-10:]:
    Plot(imgfile, np.round(score*100))
    
for imgfile, score in random.sample(img2score, 10):
    Plot(imgfile, np.round(score*100))

In [ ]:
# test2
import os

mypic_path = "../data/test_img/mypic/"
myimgfiles = [mypic_path+imgfile for imgfile in os.listdir(mypic_path)]
myimgfiles, myimgscores = get_aesthetic_score_on_batch(myimgfiles)
myimg2score = sorted(zip(myimgfiles, myimgscores), key=lambda x: x[1], reverse=True)

In [ ]:
for imgfile, score in myimg2score[-100:]:
    Plot(imgfile, np.round(score*100))

In [ ]:
for imgfile, score in myimg2score:
    if splitext(basename(imgfile))[0].startswith("20160610"):
        Plot(imgfile, np.round(score*100))